In [ ]:
# !gdown "1kfci9diSpiAUwmkuaUdt2XP9tRjo0eL2" # hpe train test
# !gdown "1Gry7E5XHJEZrqGi2WksNKmtXhLJEy9qn" # hpe song + artist
# !gdown "1qhnD4dLBEiakAvTdKygEXq_NUio3dqfu" # all song info
!gdown "1uP-aNtg7rDPDL3E5Pkkpvcl0VjZg-Qh8" # all train test + song info
!gdown "1-BopmjVlXhmMs6vS3SZzOTiDou5BWy8J" # test data

Downloading...
From: https://drive.google.com/uc?id=1uP-aNtg7rDPDL3E5Pkkpvcl0VjZg-Qh8
To: /content/hpe_all_ts.txt
100% 1.98G/1.98G [00:20<00:00, 98.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-BopmjVlXhmMs6vS3SZzOTiDou5BWy8J
To: /content/label_test_source.csv
100% 165M/165M [00:00<00:00, 187MB/s]


In [ ]:
import cupy as np  # 使用cupy可以讓array的運算使用GPU，速度可以快很多

def find_closest_vectors(vectors, target_vector, specific_vector, k=5):
    # 計算與目標向量的距離
    distances = np.linalg.norm(vectors - target_vector, axis=1)
    # 找出距離最近的 k 個向量的索引
    t1 = time.time()
    result = np.argsort(distances)[:26]
    closest_indices = []
    for i in result:
        flag = True
        for j in specific_vector:
            if np.array_equal(vectors[i], j):
                flag = False
                break
        if flag:
            closest_indices.append(i)
        if len(closest_indices) == k:
            break

    # 返回最近的 k 個向量
    return vectors[closest_indices]

In [ ]:
def find_closest_vectors_2(vectors, target_vector, specific_vector, k=5):
    # 計算與目標向量的距離
    distances = np.linalg.norm(vectors - target_vector, axis=1)

    # 將特定向量轉換為集合形式以加快查找速度
    specific_set = set(map(tuple, specific_vector))

    # 找出距離最近的 k 個向量的索引
    closest_indices = []
    for i in np.argsort(distances):
        # 使用集合來檢查是否在特定向量中
        if tuple(vectors[i]) not in specific_set:
            closest_indices.append(i)
            if len(closest_indices) == k:
                break

    # 返回最近的 k 個向量
    return vectors[closest_indices]

In [ ]:
def read_vectors_from_file(file_path): #讀song vector .txt 並處理成一個字典對照表 一個所有vectors 的array
    vectors = []
    dic = {}
    skip_first = True
    with open(file_path, 'r') as file:
        for line in file:
            if skip_first:
              skip_first = False
              continue
            # 移除空白字符，然後以逗號分隔
            vector_strings = line.strip().split(' ')

            if len(vector_strings[0]) < 10:  # not hash code
              continue
            if vector_strings[0][0:3] in ["com", "gen", "lyr", "prd", "ttl"]:  # 跳過song_id以外的內容
              continue

            # 將字符串轉換為浮點數
            vector = [float(num) for num in vector_strings[1:]]
            dic[vector_strings[0]] = vector
            # 將向量添加到列表中
            vectors.append(vector)

    return dic, np.array(vectors)

In [ ]:
def find_key_by_value(value_to_find):
  value_tuple = tuple(value_to_find)
  # 直接從反向查找字典中獲取結果
  return reverse_dic.get(value_tuple, None)

In [ ]:
import pandas as pd

def read_vectors_from_csv(file_path):
    df = pd.read_csv(file_path)
    df = df.sort_values(['session_id', 'listening_order'])
    return df.values
source_file_path = "label_test_source.csv" #test.csv 20%
source_df = read_vectors_from_csv(source_file_path)
# print(type(source_df))

In [ ]:
final_df = pd.DataFrame(columns=['session_id', 'top1', 'top2', 'top3', 'top4', 'top5'])
vector_file_path = "hpe_all_ts.txt" #放song vector .txt
dic, vectors = read_vectors_from_file(vector_file_path)

In [ ]:
import time
import numpy as nnp

dic_k = list(dic.keys())
dic_v = list(dic.values())
reverse_dic = {tuple(v): k for k, v in dic.items()}
tmp_vectors = vectors
rm_vector = []
for i in range(len(source_df)):
  # first
  if i % 20 == 0:
    session = source_df[i][0]
    source_vector = dic[source_df[i][1]]
    rm_vector.append(source_df[i][1])
  # otherwise
  else:
    source_vector = [x + y for x, y in zip(source_vector, dic[source_df[i][1]])]
    rm_vector.append(source_df[i][1])
  if i % 20 == 19:
    target_vector = [x / 20 for x in source_vector]
    # choose_vectors = remove_specific_vectors(tmp_vectors, rm_vector)
    closest_vectors = find_closest_vectors(tmp_vectors, np.array(target_vector), nnp.array(rm_vector), k=5)
    tmp_vectors = vectors
    rm_vector = []
    key = []
    closest_vectors = nnp.array(closest_vectors.get())
    for j in range(5):
      key.append(find_key_by_value(closest_vectors[j]))
    final_df.loc[i] = [session, key[0], key[1], key[2], key[3], key[4]]
  if i % 2000 == 0:
    print(i / len(source_df))

output_file_path = "/content/drive/MyDrive/datagame/output3.csv" #csv 輸出位置
final_df.to_csv(output_file_path, index=False)

0.0
0.0006989878655706537
0.0013979757311413073
0.002096963596711961
0.0027959514622826146
0.0034949393278532683
0.004193927193423922
0.004892915058994576
0.005591902924565229
0.006290890790135883
0.0069898786557065365
0.007688866521277191
0.008387854386847844
0.009086842252418499
0.009785830117989152
0.010484817983559805
0.011183805849130458
0.011882793714701113
0.012581781580271767
0.01328076944584242
0.013979757311413073
0.014678745176983728
0.015377733042554381
0.016076720908125036
0.016775708773695688
0.017474696639266343
0.018173684504836998
0.01887267237040765
0.019571660235978304
0.020270648101548955
0.02096963596711961
0.021668623832690265
0.022367611698260917
0.023066599563831572
0.023765587429402227
0.02446457529497288
0.025163563160543533
0.025862551026114188
0.02656153889168484
0.027260526757255495
0.027959514622826146
0.0286585024883968
0.029357490353967456
0.030056478219538107
0.030755466085108762
0.03145445395067942
0.03215344181625007
0.03285242968182072
0.033551417547

In [ ]:
output_file_path = "/content/drive/MyDrive/KKcompany/output.csv" #csv 輸出位置
final_df.to_csv(output_file_path, index=False)

# make all song info data (給SMORE用的)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pandas import read_parquet
meta_song_data = read_parquet("/content/drive/MyDrive/datagame/meta_song.parquet")
meta_song_composer_data = read_parquet("/content/drive/MyDrive/datagame/meta_song_composer.parquet")
meta_song_genre_data = read_parquet("/content/drive/MyDrive/datagame/meta_song_genre.parquet")
meta_song_lyricist_data = read_parquet("/content/drive/MyDrive/datagame/meta_song_lyricist.parquet")
meta_song_producer_data = read_parquet("/content/drive/MyDrive/datagame/meta_song_producer.parquet")
meta_song_titletext_data = read_parquet("/content/drive/MyDrive/datagame/meta_song_titletext.parquet")

In [ ]:
data = {"c1":[], "c2":[], "c3":[]}
padding = 0

In [ ]:
meta_song_data.fillna(-99, inplace=True)
for i in range(len(meta_song_data)):
  check_new = len(data["c1"])
  if meta_song_data["artist_id"][i] != -99:
    data["c1"].append(meta_song_data["song_id"][i])
    data["c2"].append("art" + str(int(meta_song_data["artist_id"][i])))
  if meta_song_data["album_id"][i] != -99:
    data["c1"].append(meta_song_data["song_id"][i])
    data["c2"].append("alb" + str(int(meta_song_data["album_id"][i])))
  if meta_song_data["language_id"][i] != -99:
    data["c1"].append(meta_song_data["song_id"][i])
    data["c2"].append("lan" + str(int(meta_song_data["language_id"][i])))

  if len(data["c1"]) == check_new:  # 沒有新增資料的話，至少要新增song_id
    data["c1"].append(meta_song_data["song_id"][i])
    data["c2"].append("nul" + str(padding))
    padding += 1

In [ ]:
meta_song_composer_data.fillna(-99, inplace=True)
for i in range(len(meta_song_composer_data)):
  data["c1"].append(meta_song_composer_data["song_id"][i])
  data["c2"].append("com" + str(meta_song_composer_data["composer_id"][i]))

In [ ]:
meta_song_genre_data.fillna(-99, inplace=True)
for i in range(len(meta_song_genre_data)):
  data["c1"].append(meta_song_genre_data["song_id"][i])
  data["c2"].append("gen" + str(meta_song_genre_data["genre_id"][i]))

In [ ]:
meta_song_lyricist_data.fillna(-99, inplace=True)
for i in range(len(meta_song_lyricist_data)):
  data["c1"].append(meta_song_lyricist_data["song_id"][i])
  data["c2"].append("lyr" + str(meta_song_lyricist_data["lyricist_id"][i]))

In [ ]:
meta_song_producer_data.fillna(-99, inplace=True)
for i in range(len(meta_song_producer_data)):
  data["c1"].append(meta_song_producer_data["song_id"][i])
  data["c2"].append("prd" + str(meta_song_producer_data["producer_id"][i]))

In [ ]:
# meta_song_titletext_data.fillna(-99, inplace=True)
# for i in range(len(meta_song_titletext_data)):
#   data["c1"].append(meta_song_titletext_data["song_id"][i])
#   data["c2"].append("ttl" + str(meta_song_titletext_data["title_text_id"][i]))

In [ ]:
data["c3"] = [1] * len(data["c1"])

In [ ]:
import pandas as pd

In [ ]:
pd.DataFrame(data).to_csv("/content/drive/MyDrive/datagame/song_all_info.csv", index=False)